<a href="https://colab.research.google.com/github/pradipNP/DecisionTree-Iris/blob/main/8x_puzzle_assign.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
class Node:
    def __init__(self, data, level, fval):

        """Initialize the node with data, level, and calculated f-value.
        :param data: The state of the puzzle (2D list)
        :param level: The depth level of this node in the search tree
        :param fval: The calculated f-value (heuristic + cost)"""

        self.data = data
        self.level = level
        self.fval = fval

    def generate_child(self):

        """Generate child nodes by moving the blank space in four directions: up, down, left, right.
        :return: A list of child nodes."""

        x, y = self.find(self.data, '_')
        val_list = [[x, y - 1], [x, y + 1], [x - 1, y], [x + 1, y]]
        children = []

        for i in val_list:
            child_data = self.shuffle(self.data, x, y, i[0], i[1])
            if child_data is not None:
                child_node = Node(child_data, self.level + 1, 0)
                children.append(child_node)

        return children

    def shuffle(self, puz, x1, y1, x2, y2):
        """
        Swap the blank space with the target position, if within bounds.
        :return: The new puzzle state (2D list) or None if out of bounds.
        """
        if 0 <= x2 < len(self.data) and 0 <= y2 < len(self.data):
            temp_puz = self.copy(puz)
            temp_puz[x2][y2], temp_puz[x1][y1] = temp_puz[x1][y1], temp_puz[x2][y2]
            return temp_puz
        return None

    @staticmethod
    def copy(root):
        """
        Create a copy of the puzzle state.
        :return: A copy of the given puzzle (2D list).
        """
        return [row[:] for row in root]

    @staticmethod
    def find(puz, val):
        """
        Find the position of a value (e.g., blank space '_') in the puzzle.
        :return: The row and column index of the value.
        """
        for i in range(len(puz)):
            for j in range(len(puz)):
                if puz[i][j] == val:
                    return i, j


class Puzzle:
    def __init__(self, size):
        """
        Initialize the puzzle size, open list, and closed list.
        :param size: The size of the puzzle (e.g., 3 for a 3x3 puzzle).
        """
        self.n = size
        self.open = []
        self.closed = []

    def accept(self):
        """
        Accept a puzzle state as input from the user.
        :return: The puzzle state (2D list).
        """
        print(f"Enter the {self.n}x{self.n} puzzle row by row (use '_' for the blank space):")
        puz = []
        for i in range(self.n):
            row = input(f"Row {i + 1}: ").strip().split()
            puz.append(row)
        return puz

    def f(self, start, goal):
        """
        Calculate the f-value: f(x) = h(x) + g(x).
        :return: The f-value.
        """
        return self.h(start.data, goal) + start.level

    def h(self, start, goal):
        """
        Calculate the heuristic value (number of misplaced tiles).
        :return: The heuristic value.
        """
        temp = 0
        for i in range(self.n):
            for j in range(self.n):
                if start[i][j] != goal[i][j] and start[i][j] != '_':
                    temp += 1
        return temp

    def process(self):
        """
        Accept start and goal states, then solve the puzzle.
        """
        print("Enter the start state:")
        start = self.accept()
        print("\nEnter the goal state:")
        goal = self.accept()

        start_node = Node(start, 0, 0)
        start_node.fval = self.f(start_node, goal)
        self.open.append(start_node)

        print("\nSolving the puzzle...\n")
        while self.open:
            current_node = self.open[0]

            # Display the current node
            print("Step:")
            for row in current_node.data:
                print(" ".join(row))
            print("\n")

            # If the current state matches the goal state, we've solved it
            if self.h(current_node.data, goal) == 0:
                print("Puzzle solved!")
                return

            # Generate child nodes and calculate their f-values
            for child in current_node.generate_child():
                child.fval = self.f(child, goal)
                self.open.append(child)

            # Move the current node to the closed list
            self.closed.append(current_node)
            del self.open[0]

            # Sort the open list based on f-values (lowest first)
            self.open.sort(key=lambda x: x.fval)


# Run the puzzle solver
if __name__ == "__main__":
    size = 3  # For a 3x3 puzzle
    puzzle = Puzzle(size)
    puzzle.process()


Enter the start state:
Enter the 3x3 puzzle row by row (use '_' for the blank space):
Row 1: 1 2 3
Row 2: 5 6 _
Row 3: 7 8 4

Enter the goal state:
Enter the 3x3 puzzle row by row (use '_' for the blank space):
Row 1: 1 2 3
Row 2: 5 8 6
Row 3: _ 7 4

Solving the puzzle...

Step:
1 2 3
5 6 _
7 8 4


Step:
1 2 3
5 _ 6
7 8 4


Step:
1 2 3
5 8 6
7 _ 4


Step:
1 2 3
5 8 6
_ 7 4


Puzzle solved!
